In [2]:
import logging
import logging.config
import os
import json
# import ConfigParser
import sys
import re
from glob import glob

import configuration
from gdrive.auth import getCredentials
from gdrive.gdrive import googledrive, GDriveError

from humanfriendly import prompts

# get the current working directory
# When launching a .command from the OS X Finder, the working directory is typically ~/; this is problematic
# for locating resource files
# I don't love this hack, but it works.
try:
    __file__
    cwd = os.path.dirname(__file__)+'/'
except NameError as e:
    cwd = os.getcwd()

In [3]:
def setup_logging(
    default_path='logging.json',
    default_level=logging.INFO,
    env_key='LOG_CFG'
):
    """Setup logging configuration

    """
    path = default_path
    value = os.getenv(env_key, None)
    if value:
        path = value
    if os.path.exists(path):
        with open(path, 'rt') as f:
            config = json.load(f)
        logging.config.dictConfig(config)
        logging.getLogger().setLevel(default_level)
    else:
        logging.basicConfig(level=default_level)

In [4]:
def fileSearch(path = None, search = None):
        '''search for a file name string in a given path'''
        if not search:
            search = ''
        logger = logging.getLogger(__name__)
        logger.debug('searching path: {0} for \"{1}\"'.format(path, search))
        try:
            searchPath = os.path.expanduser(path)
        except AttributeError as e:
            logger.error('Error: no path specified')
            searchPath = ''
        try:
            allFiles = os.listdir(searchPath)
        except OSError as e:
            logger.error(e)
            return(None)
        
        # list comprehension search with regex
        #http://www.cademuir.eu/blog/2011/10/20/python-searching-for-a-string-within-a-list-list-comprehension/
        regex = re.compile('.*('+search+').*')
        return([m.group(0) for l in allFiles for m in [regex.search(l)] if m])

In [5]:
def getConfiguration(cfgfile):
    # required configuraiton options
    # Section: {'option': 'default value'}
    cfgpath = os.path.dirname(cfgfile)
    config_required = {
        'Main': {'credentials': os.path.join(cfgpath, 'credentials/'), 
                 'teamdriveID': '',
                 'teamdrivename': '',
                 'folderID': '',
                 'foldername': '',
                 },
        }

    config = configuration.get_config(cfgfile)

    update_config = False

    for section, values in config_required.items():
        if not config.has_section(section):
            logger.warn('section: {} not found in {}'.format(section, cfgfile))
            logger.debug('adding section {}'.format(section))
            config.add_section(section)
            update_config = True
        for option, value in values.items():
            if not config.has_option(section, option):
                logger.warn('option: {} not found in {}'.format(option, cfgfile))
                logger.debug('adding option {}: {}'.format(option, value))

                config.set(section, option, value)
                update_config = True


    # for section, options in config_required.items():

    if update_config:
        try:
            logger.info('updating configuration file at: {}'.format(cfgfile))
            configuration.create_config(cfgfile, config)
        except Exception as e:
            logger.error(e)
            
    return(config)

In [6]:
def getTeamDrive(myDrive):
    '''
    menu driven interaction for choosing a writeable team drive
    '''
    #available and writeable team drives
    logger = logging.getLogger(__name__)
    try:
        teamdrives_all = myDrive.listTeamDrives()
    except Exception as e:
        logger.error(e)
    
    if len(teamdrives_all) < 1:
        logger.warn('no writeable team drives found for this user')
        return(None)
    
    teamdrives_writeable = {}
    for drive in teamdrives_all:
        if drive['capabilities']['canAddChildren']:
            teamdrives_writeable[drive['name']] = drive
    
    teamdrive = None
    
    try:
        teamdrive_name = prompts.prompt_for_choice(choices=teamdrives_writeable, default=None)
        teamdrive=teamdrives_writeable[teamdrive_name]
    except prompts.TooManyInvalidReplies as e:
        logger.warning('user failed to make a choice')
    
    return(teamdrive)

In [7]:
def getPortfolioFolder(myDrive):
    '''
    menu driven interaction to locate and choose portfolio folder on team drive
    
    returns:
        folder (dictionary) - google drive object properties dictionary
    '''
    logger = logging.getLogger(__name__)
    logger.info('starting search for portfolio folder in teamdrive: {}; {}'.format(teamdriveName, teamdriveID))
    
    foldersearch = ''
    attempt = 5
    while (len(foldersearch) <= 0) and (attempt > 0):
        attempt -= 1
        foldersearch = prompts.prompt_for_input(question='Please enter a portion of the portfolio folder name (case insensitive): ',
                                                default=None, strip=True)
        if len(foldersearch) <= 0:
            print 'Nothing entered. {} attempts remain.'.format(attempt)
            
        searchresult = myDrive.search(name=foldersearch, fuzzy=True, teamdrive = teamdriveID, mimeType='folder')
        if len(searchresult['files']) <= 0:
            print 'No folders contained "{}"'.format(foldersearch)
            print 'Please try entering only part of the folder name'
            foldersearch = ''
            
    
    
    if foldersearch <= 0:
        logger.warn('user entered 0 length string for folder search')
        return(None)
    
    
    foldernames = []
    for each in searchresult['files']:
        foldernames.append(each['name'])
    
    try:
        foldername = prompts.prompt_for_choice(choices=foldernames)
    except prompts.TooManyInvalidReplies as e:
        logger.warning('user failed to make a choice')
        return(None)
    
    return(searchresult['files'][foldernames.index(foldername)])
    

In [8]:
def getPathfromList(list_path=['~/'], message='Choose from the paths below', default=None):
    '''
    menu driven interaction to select path 
    accepts:
        list_path (list) - list of paths to display
        messager (string) - message to display
        default (string) - default choice (must be one of the items in the list otherwise defaults to None)
    
    returns:
        searchPath (string) - valid and existing path
    '''
    logger = logging.getLogger(__name__)
    logger.debug('path list: {}'.format(list_path))
    searchPath = None
    list_path = sorted(list_path)
    list_path.append('OTHER')
    
    if default not in list_path:
        logging.warn('default ({}) not in list; setting default = None'.format(default))
    
    if message:
        print message
    searchPath = prompts.prompt_for_choice(choices=list_path, default=default)
    logger.debug('searchPath = {}'.format(searchPath))
    
    if not searchPath is 'OTHER':
        searchPath = os.path.expanduser(searchPath)
    else:
        attempt = 5
        searchPath = None
        while not searchPath and attempt > 0:
            logger.debug('attempts remaining: {}'.format(attempt))
            attempt -= 1
            searchPath = prompts.prompt_for_input('Please type the complete path to use: ')
            logger.debug('searchPath = {}'.format(searchPath))         
    
    if len(searchPath) < 1:
        searchPath = None
    return (searchPath) 

In [20]:
def getFilefromPath(path='~/', pattern='*', ignorecase=True):
    '''
    search path for files matching pattern (regular expression)
    accepts:
        path (string) - path to search
        pattern (string) - regular expression to search for in file names
        ignorecase (bool) - defaults to True, ignore file name case'''
    logger = logging.getLogger(__name__)
    
    path = os.path.expanduser(path+'/*')
    logger.debug('Path: {}, glob pattern: {}'.format(path, pattern))
    
    flags = re.IGNORECASE
    files = []
        
    if ignorecase:
        for eachfile in [f for f in glob(path) if re.match(pattern, f, flags=flags)]:
            files.append(eachfile)
    
    if not ignorecase:
        for eachfile in [f for f in glob(path) if re.match(pattern, f)]:
            files.append(eachfile)        
    
    return(files)

In [21]:
getFilefromPath('~/Downloads', '.*Student.*export.*', ignorecase = False)

[]

In [39]:
import fnmatch
files = []
for eachfile in os.listdir('/Users/aaronciuffo/Downloads/'):
    if fnmatch.fnmatch(eachfile, '*tudent*'):
        print eachfile

student.export (1).text
student.export.text
student_export.text
Student_Export.txt


In [10]:
version = '00.00 - 18.08.13'
appName = 'portfolio_creator'

cfgfile = appName+'.ini'
cfgpath = os.path.join('~/.config/', appName)
cfgfile = os.path.expanduser(os.path.join(cfgpath, cfgfile))
studentFile = None

downloadlink = 'https://foo.foof.foo'


updateConfig = False


logger = logging.getLogger(__name__)
setup_logging(default_level=logging.DEBUG)
logging.info('===Starting {} Log==='.format(appName))

myConfig = getConfiguration(cfgfile)

teamdriveName = '**UNKNOWN**'

####### set all config options 
if myConfig.has_option('Main', 'credentials'):
    credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))
else:
    credential_store = os.path.expanduser(os.path.join(cfgpath, 'credentials'))
    updateConfig = True

if myConfig.has_option('Main', 'teamdriveid'):
    teamdriveID = myConfig.get('Main', 'teamdriveid')
else:
    teamdriveID = None
    
if myConfig.has_option('Main', 'teamdrivename'):
    teamdriveName = myConfig.get('Main', 'teamdrivename')
else:
    teamdriveName = None
    
if myConfig.has_option('Main', 'folderid'):
    folderID = myConfig.get('Main', 'folderid')
else:
    folderID = None

if myConfig.has_option('Main', 'foldername'):
    folderName = myConfig.get('Main', 'foldername')
else:
    folderName = None

if myConfig.has_option('Main', 'gradefolders'):
    gradefolders = myConfig.get('Main', 'gradefolders')
else:
    gradefolders = './gradefolders.txt'


    
if logging.getLogger().getEffectiveLevel() <= 10:
    config_dict = {}
    for section in myConfig.sections():
        config_dict[section] = {}
        for option in myConfig.options(section):
            config_dict[section][option] = myConfig.get(section, option)

    logger.debug('current configuration:')
    logger.debug('\n{}'.format(config_dict))

# google drive authorization
logging.info('checking google credentials')
#### Add instructions before this shoots user over to a web authorization 
print '\n'*3
print 'You *may* be directed to a google web page that asks you to authorize this applicaiton.'
print 'Please choose an ASH account and authorize this applicaiton.'
print 'When you are done, please close the newly created tab in your web browser and return to this window.'
if not prompts.prompt_for_confirmation(question='Would you like to proceed?', default='y'):
    print 'Exiting'
    exit(0)
    
credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))

# attempt to authorize using stored credentials or generate new credentials as needed
try:
    credentials = getCredentials(credential_store)
except Exception as e:
    logging.critical(e)

# configure google drive object
myDrive = googledrive(credentials)

if not teamdriveID:
    logger.warn('Team Drive ID missing')
    try:
        teamdrive = getTeamDrive(myDrive)
    except GDriveError as e:
        logging.error('Error accessing team drives: e')
        exit(0)
        
    if not teamdrive:
        logger.error('no team drives found')
        logger.error('userinfo: {}'.format(myDrive.userinfo['emailAddress']))
        print('No Team Drives located; cannot proceed.')
        print('You are authenticated as: {}'.format(myDrive.userinfo['emailAddress']))
        logger.error('exiting')
        exit(0)
        
    try:
        teamdriveID = teamdrive['id']
        teamdriveName = teamdrive['name']
    except (KeyError, TypeError) as e:
        logger.error('no valid team drive information found: {}'.format(e))
        logger.error('userinfo: {}'.format(myDrive.userinfo['emailAddress']))
        print 'No Team Drive was found. Do you have write access to a Team Drive with this account?'
        print 'You are authenticated as: {}'.format(myDrive.userinfo['emailAddress'])
        logger.error('exiting')
        exit(0)
        
    print 'Using Team Drive: {}'.format(teamdriveName)    
    myConfig.set('Main', 'teamdriveid', teamdriveID)
    myConfig.set('Main', 'teamdriveName', teamdriveName)
    updateConfig = True
    
if not folderID:
    logger.warn('Folder ID missing')
    try:
        folder = getPortfolioFolder(myDrive)
    except GDriveError as e:
        logger.error('Could not search for folder: {}'.format(e))
        print ('There was a problem searching Team Drive for a folder')
        print ('You are authenticated as: {}'.format(myDrive.userinfo['emailAddress']))
        print ('If this is not correct do...')
    try:
        folderID = folder['id']
        folderName = folder['name']
    except (KeyError, TypeError) as e:
        logger.error('no valid folder information found {}'.format(e))
        logger.error('exiting')
        exit(0)
    
    print 'Using Folder: {} on Team Drive: {}'.format(folderName, teamdriveName)
    myConfig.set('Main', 'foldername', folderName)
    myConfig.set('Main', 'folderid', folderID)
    updateConfig=True

if not os.path.exists(gradefolders):
    logger.error('failed to locate {}'.format(gradefolders))
    logger.error('exiting')
    print 'gradefolders.txt file is missing. Please reintall the application.'
    print 'download link: {}'.format(downloadlink)
    exit(0)

# get the gradefolder file
list_gradefolder = []
with open(gradefolders, 'r') as fhandle:
    logger.debug('reading gradefolders: {}'.format(gradefolders))
    for line in fhandle:
        list_gradefolder.append(line.strip('\n'))
    

proceed = False
while not proceed:
    print 'current configuration:'
    print ''
# get and validate path for student_export folder


print '\n'*3
print ''


if updateConfig:
    configuration.create_config(cfgfile, myConfig)


2018-09-12 18:48:28,571: [INFO: root.<module>] ===Starting portfolio_creator Log===
2018-09-12 18:48:28,572: [DEBUG: configuration.get_config] reading configuration file at: /Users/aaronciuffo/.config/portfolio_creator/portfolio_creator.ini
2018-09-12 18:48:28,574: [DEBUG: __main__.<module>] current configuration:
2018-09-12 18:48:28,575: [DEBUG: __main__.<module>] 
{'Main': {'folderid': '1NWVX-ZDTbT5yUT-Rw3a3fHQNptB4W4lb', 'credentials': '~/.config/portfolio_creator/credentials/', 'foldername': 'Portfolios', 'teamdrivename': 'IT Blabla', 'teamdriveid': '0ACLfU8KeD_BHUk9PVA'}}
2018-09-12 18:48:28,576: [INFO: root.<module>] checking google credentials




You *may* be directed to a google web page that asks you to authorize this applicaiton.
Please choose an ASH account and authorize this applicaiton.
When you are done, please close the newly created tab in your web browser and return to this window.
2018-09-12 18:48:28,577: [DEBUG: humanfriendly.prompts.prompt_for_confirmation] Request

2018-09-12 18:48:36,786: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/about?fields=user&alt=json
2018-09-12 18:48:36,944: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/teamdrives?fields=teamDrives&alt=json
2018-09-12 18:48:37,503: [DEBUG: __main__.<module>] reading gradefolders: ./gradefolders.txt







In [12]:
getPathfromList(['~/Downloads', '~/Desktop'], 
                message='Please select location for the student_export.text file to use.',
                default='~/Downloads')

2018-09-12 18:51:57,727: [DEBUG: __main__.getPathfromList] path list: ['~/Downloads', '~/Desktop']
Please select location for the student_export.text file to use.
2018-09-12 18:51:57,728: [DEBUG: humanfriendly.prompts.prompt_for_choice] Requesting interactive choice on terminal (options are '~/Desktop', '~/Downloads' and 'OTHER') ..

  1. ~/Desktop
  2. ~/Downloads (default choice)
  3. OTHER
 
 Enter your choice as a number or unique substring (Control-C aborts): 
2018-09-12 18:52:03,344: [DEBUG: humanfriendly.prompts.prompt_for_choice] Default choice selected by empty reply ('~/Downloads').
2018-09-12 18:52:03,345: [DEBUG: __main__.getPathfromList] searchPath = ~/Downloads


'/Users/aaronciuffo/Downloads'

In [ ]:
for each in ['~/Downloads', '~/Desktop', 'OTHER']:
    print os.path.expanduser(each)

In [ ]:
mylist = ['~/Downloads', '~/Desktop']

In [ ]:
sorted(mylist)

In [ ]:
getStudentExportPath()

In [ ]:
def getStudentExportFile(exportpath):
    '''
    menu driven interaction to read and validate student_export.text file
    '''

In [ ]:
fileSearch('~/Downloads', 'student*export')

In [ ]:
foo = getStudentExport()

In [ ]:
myDrive.userinfo

In [ ]:
print list_gradefolder

In [ ]:
print gradefolders

In [ ]:
def xgetTeamDrive(myDrive):
    '''
    menu interaction to find appropriate teamdrive ID for storing portfolios
    '''
    logger = logging.getLogger(__name__)
    myMenu = Menu()
    teamDrives = {}
    logging.debug('fetching list of team drives')
    for each in myDrive.listTeamDrives():
        try:
            teamDrives[each['name']] = each['id']
            myMenu.addItem(each['name'])
        except KeyError as e:
            logger.warn('KeyError: {}'.format(e))
            
    myMenu.sortMenu()
    td_name = myMenu.loopChoice(optional=True, message='Choose Team Drive for Portfolios')
    if td_name.upper() == 'Q':
        return('Q', 'Q')
    else:
        return([td_name, teamDrives[td_name]])

In [ ]:
def xgetPortfolioFolder(myDrive, teamDriveID):
    # ask user for partial folder name
        # check for non 0 input
    # if results are returned ask user to choose from list, or search again
    # if no results are returned, prompt user for new search terms
    # return the name and id of the folder
    
    logger = logging.getLogger(__name__)
    logger.info('starting search for portfolio folder in teamdrive ID: {}'.format(teamdriveID))
    print 'Locating Portfolio Folder in Team Drive'
    
    search = True
    attempts = 3
    
    quitMenu = Menu(items=['Yes', 'No'])
    
    while search:
        logger.debug('remaining attempts: {}'.format(attempts))
        attempts -=1
        foldersearch = ''
        folderID = ''
        folderName = ''
        folderlist = {}
        myMenu = Menu()
        
        # get a partial file name to search for
        while len(foldersearch) <= 0:
            foldersearch = raw_input('Enter part of the portfolio folder name: ')
            if len(foldersearch) <=0:
                print 'Please enter at least one character.'
                
        # fetch results from google drive
        logger.debug('searching for folder name: {}'.format(foldersearch))
        searchresult = myDrive.search(name=foldersearch, fuzzy=True, teamdrive = teamdriveID, mimeType='folder')
        try:
            rawfilelist = searchresult['files']
        except KeyError as e:
            rawfilelist = {}
            logger.warn(e)
    
        if len(rawfilelist) < 1:
            print 'No matching folders were found. Please check your spelling and try a new search.'
            logger.debug('no matching folders found')
        else:
            for each_folder in rawfilelist:
                folderlist[each_folder['name']] = each_folder['id']
                myMenu.addItem(each_folder['name'])
                
            myMenu.sortMenu()
            folderName = myMenu.loopChoice()
            folderID = folderlist[folderName]
            if len(folderID) > 0:
                logger.debug('exiting folder choice loop')
                break
        
            
            
        # ask to break out of loop after attempts reaches 0
        if attempts <= 0:
            quit = quitMenu.loopChoice(message='It looks like you\'re having trouble. Would you like to try again?')
            if quit == 'No':
                search = False
            else:
                attempts = 3
        
    
    
    
    return(folderName, folderID)

In [ ]:
def xgetStudentFile():
    '''
    menu interaction to choose appropriate student_export.text file to import
    Returns:
        studentFile (string): path to student_export.text file
    '''
    logger = logging.getLogger(__name__)
    items = ['Desktop', 'Downloads', 'Documents']
    fileMenu = Menu(name='Location to search for "student_export file',items=items)
    fileMenu.sortMenu()
    studentFolder = fileMenu.loopChoice(optional=True,
                                        message='Please specify the location of a PowerSchool "student_export" file')
    if studentFolder == 'Q':
        logger.debug('user quit')
        return('Q')
    
    searchPath = os.path.expanduser('~/'+studentFolder)
    fileList = fileSearch(searchPath, 'text')
    logger.debug('files found: {}'.format(len(fileList)))
    if fileList is None:
        print '{0} folder does not exist!'.format(searchPath)
        return(None)
    elif len(fileList) < 1:
            print '{0} folder does not contain any files that end with ".text"\n'.format(searchPath)
            return(None)
    
    fileMenu = Menu(name = 'located student_export files', items = fileList)
    fileMenu.sortMenu()
    
    myFile = fileMenu.loopChoice(optional=True, message='Please choose the appropriate student_export file to import')
    
    if myFile == 'Q':
        logger.debug('user quit')
        return('Q')
    
    return(os.path.join(searchPath, myFile))
    

In [ ]:
def askContinue(studentExport, myConfig={}):
    '''
    menu interaction ask if everything is correct and run the folder population
    '''
    logger = logging.getLogger(__name__)
    logger.debug('checking configuration with user')
    continueMenu = Menu(name='Continue', items=['Yes', 'No'] ) 
    choice = None
    # print the configuration here
    cfgDisplay = ['teamdrivename', 'foldername', 'gradefolders', 'studenexport']
    print 'These are the settings that will be used to create portfolio folders:'
    for option in cfgDisplay:
        try:
            print '{}: {}'.format(option, myConfig.get('Main', option))
        except Exception as e:
            logger.debug(e)
    print 'studentexport: {}\n'.format(studentExport)
        
    
    while not choice:
        choice = continueMenu.loopChoice(optional=True, message='Begin folder creation with the settings shown above?')

        if choice == 'No':
            changeMenu = Menu(items=cfgDisplay, name='Current Configuration')
            change = changeMenu.loopChoice(optional=True, message='Which option would you like to change?')
            if change == 'teamdrivename':
                
            choice = None


        if choice == 'Yes':
            return('Y')

        if choice == 'Q':
            return('Q')
    
    return(choice)
    

In [ ]:
askContinue('foo', myConfig)

In [ ]:
version = '00.00 - 18.08.13'
appName = 'portfolio_creator'

cfgfile = appName+'.ini'
cfgpath = os.path.join('~/.config/', appName)
cfgfile = os.path.expanduser(os.path.join(cfgpath, cfgfile))
studentFile = None
updateConfig = False


logger = logging.getLogger(__name__)
setup_logging(default_level=logging.DEBUG)
logging.info('===Starting {} Log==='.format(appName))

myConfig = getConfiguration(cfgfile)

####### set all config options 
if myConfig.has_option('Main', 'credentials'):
    credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))
else:
    credential_store = os.path.expanduser(os.path.join(cfgpath, 'credentials'))
    updateConfig = True

if myConfig.has_option('Main', 'teamdriveid'):
    teamdriveID = myConfig.get('Main', 'teamdriveid')
else:
    teamdriveID = None
    
if myConfig.has_option('Main', 'teamdrivename'):
    teamdriveName = myConfig.get('Main', 'teamdrivename')
else:
    teamdriveName = None
    
if myConfig.has_option('Main', 'folderid'):
    folderID = myConfig.get('Main', 'folderid')
else:
    folderID = None

if myConfig.has_option('Main', 'foldername'):
    folderName = myConfig.get('Main', 'foldername')
else:
    folderName = None

if myConfig.has_option('Main', 'gradefolders'):
    gradefolders = myConfig.get('Main', 'gradefolders')
else:
    gradefolders = None
    
if logging.getLogger().getEffectiveLevel() <= 10:
    config_dict = {}
    for section in myConfig.sections():
        config_dict[section] = {}
        for option in myConfig.options(section):
            config_dict[section][option] = myConfig.get(section, option)

    logging.debug('current configuration:')
    logging.debug('\n{}'.format(config_dict))

# print '\n'*10
print 'Welcome to the portfolio creator for Google Team Drive version {0}'.format(version)
print 'This program will create portfolio folders in Google Team Drive for students.'
print 'You will need a student_export.text file from PowerSchool with at least the following information:'
print '     ClassOf, FirstLast, Student_Number\n'
print 'The order of the CSV does not matter, but the headers must be on the very first line.'
# raw_input("Press Enter to continue...\n\n\n")

    
# google drive authorization
logging.info('checking google credentials')
#### Add instructions before this shoots user over to a web authorization 
print '\n'*3
print 'You *may* be directed to a google web page that asks you to authorize this applicaiton.'
print 'Please choose an ASH account and authorize this applicaiton.'
print 'When you are done, please close the newly created tab in your web browser and return to this window.'
raw_input("Press Enter to continue...\n\n\n")
credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))

# attempt to authorize using stored credentials or generate new credentials as needed
try:
    credentials = getCredentials(credential_store)
except Exception as e:
    logging.critical(e)

# configure google drive object
myDrive = googledrive(credentials)

if (not teamdriveID) or (len(teamdriveID)< 1):
    teamdriveName, teamdriveID = getTeamDrive(myDrive)
    if teamdriveID == 'Q':
        logging.debug('user exit')
        sys.exit(0)
    myConfig.set('Main', 'teamdriveid', teamdriveID)
    myConfig.set('Main', 'teamdrivename', teamdriveName)
    updateConfig = True


if not folderID or len(folderID) < 1:
    folderName, folderID = getPortfolioFolder(myDrive, teamdriveID)
    if len(folderID) < 1:
        logging.debug('user exit')
        print 'exiting'
    myConfig.set('Main', 'folderID', folderID)
    myConfig.set('Main', 'folderName', folderName)
    updateConfig = True


if not gradefolders or len(gradefolders) < 1:
    logger.debug('searching cwd for gradefolders')
    if os.path.isfile('./gradefolders.txt'):
        logger.debug('gradefolders found in cwd')
        gradefolders = './gradefolders.txt'
        myConfig.set('Main', 'gradefolders', gradefolders)
        updateConfig = True   
    # if a grade folder on the desktop exists, use this one
    elif os.path.isfile(os.path.expanduser('~/Desktop/gradefolders.txt')):
        logger.debug('found gradefolders on ~/Desktop, using')
        gradefolders = os.path.expanduser('~/Desktop/gradefolders.txt')
    else:
        print 'The gradefolders.txt file appears to be missing.'
        print 'gradefolders.txt should contain the name of one grade level per line'
        print 'staring with Preeschool through 12th grade in the format below:'
        print '\n00-Preeschool\n00-Transition Kindergarten\n00-zKindergarten...\n09-Grade\n10-Grade\n11-Grade\n12-Grade'
        print 'please create a gradefolders.txt file and place it on the desktop'
        print 'exiting'
        sys.exit(0)
        

while not studentFile:
    studentFile = getStudentFile()
    if studentFile == 'Q':
        print 'exiting'
        sys.exit(0)
logger.debug('student file chosen: {}'.format(studentFile))



    
if updateConfig:
    configuration.create_config(cfgfile, myConfig)


